## Quick Start: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform


In [31]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [32]:
df = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")

df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [33]:
# split

train,test = train_test_split(df, test_size=0.25, random_state=42)

In [34]:
X_train = train.drop(["quality"], axis=1).values
y_train = train[["quality"]].values.ravel()
X_test = test.drop(["quality"], axis=1).values
y_test = test[["quality"]].values.ravel()

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)


In [35]:
# ANN model

def train_model(params, epochs, X_train, y_train, X_val, y_val, X_test, y_test):
    # Define model architecture

    # Use for normalization
    mean = np.mean(X_train, axis=0)
    var = np.var(X_train, axis=0)
    
    model = keras.Sequential(
        [
            keras.Input(shape=([X_train.shape[1]])),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    model.compile(optimizer=keras.optimizers.SGD(
                    learning_rate=params["lr"],
                    momentum=params["momentum"]
                  ),
                  loss="mean_squared_error",
                  metrics=[keras.metrics.RootMeanSquaredError()]
                  )
    
    with mlflow.start_run(nested = True):
        # Train model
        model.fit (X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), batch_size=64)

        eval_result = model.evaluate(X_val, y_val, batch_size=64)
        eval_rmse = eval_result[1]

        # Log the params and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log the model

        mlflow.tensorflow.log_model(model, "model", signature = infer_signature(X_train, y_train))

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [36]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        X_train=X_train,
        y_train=y_train,
        X_val=X_val,
        y_val=y_val,
        X_test=X_test,
        y_test=y_test,
    )
    return result

In [37]:
space={
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [38]:
mlflow.set_experiment("wine-quality")

with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials,
    )

    # Fetch the best result
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log best params
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature = infer_signature(X_train, y_train))

    # Print out the best parameters and corresponding loss
    print("Best parameters were: ", best)
    print("Best loss was: ", best_run["loss"])

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - loss: 34.8681 - root_mean_squared_error: 5.9049
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.8568 - root_mean_squared_error: 2.4517 - val_loss: 1.0055 - val_root_mean_squared_error: 1.0028

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.2137 - root_mean_squared_error: 1.1017
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8889 - root_mean_squared_error: 0.9421 - val_loss: 0.6861 - val_root_mean_squared_error: 0.8283

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.7494 - root_mean_squared_error: 0.8657
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6387 - root_mean_squared_error: 0.7990 - val_loss: 0.5938 - val_root_mean_squared_error: 0.7706

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.5297 - root_mean_squared_error: 0.7278
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [39]:
# Do Inferencing
# mlfow ui validation code
from mlflow.models import validate_serving_input

model_uri = 'runs:/c8422bb669374cc381b8f671118a67d7/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(X_test)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

/Users/kyuho/Codes/learn-mlflow/MlFlowStarter/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 1/39 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[5.915736 ],
       [6.524808 ],
       [6.6552134],
       ...,
       [6.31748  ],
       [6.8568296],
       [6.0198812]], dtype=float32)

In [40]:
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[5.915736 ],
       [6.524808 ],
       [6.6552134],
       ...,
       [6.31748  ],
       [6.8568296],
       [6.0198812]], dtype=float32)

In [41]:
# Register in the model registry
mlflow.register_model(model_uri, "wine-quality-model")

Successfully registered model 'wine-quality-model'.
Created version '1' of model 'wine-quality-model'.


<ModelVersion: aliases=[], creation_timestamp=1736267104720, current_stage='None', description=None, last_updated_timestamp=1736267104720, name='wine-quality-model', run_id='c8422bb669374cc381b8f671118a67d7', run_link=None, source='file:///Users/kyuho/Codes/learn-mlflow/MlFlowStarter/2-DLMLFLOW/mlruns/813730235102080139/c8422bb669374cc381b8f671118a67d7/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>